# 활성화 데이터 추출코드


In [358]:
import pandas as pd

data_origin: object = pd.read_excel('data/활성화_원본.xlsx',sheet_name = 5)
data_origin = pd.DataFrame(data_origin)
data_origin

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,NaN,"■법인별 주요 불량 구분 (공정손실비용 - Sample, 샘플, QA 검사 제외)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,구분,NaN,NaN,NaN,21년,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,1월,NaN,2월,NaN,3월,...,9월,NaN,10월,NaN,11월,NaN,12월,NaN,21年 Total,NaN
4,NaN,자동차\n활성화,Site,불량 구분,NaN,율\n(%),공정손실\n비용(억원),율\n(%),공정손실\n비용(억원),율\n(%),...,율\n(%),공정손실\n비용(억원),율\n(%),공정손실\n비용(억원),율\n(%),공정손실\n비용(억원),율\n(%),공정손실\n비용(억원),율\n(%),공정손실\n비용(억원)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,NaN,NaN,NaN,NaN,특성측정 Taping(%),4.95e-06,0.00385412,2.58083e-07,0.000201977,1.68895e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.467379
302,NaN,NaN,NaN,NaN,특성측정 Sealing 주름(%),0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
303,NaN,NaN,NaN,NaN,특성측정 Cold IR(%),NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
304,NaN,NaN,NaN,NaN,특성측정 절연 저항(%),NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.25649


데이터 추출 / 1차 필터링

In [359]:
# 공정데이터 가져옴 -> 유효row 인덱스만 가져옴 --> 전체 데이터에서 유효 인덱스만 추출해서 따로 저장(filtered)

# 공정데이터가 있는 열을 가져오고 nan값 제거
data_process = pd.DataFrame(data_origin["Unnamed: 4"])
data_process = data_process.dropna()

# 유효row의 인덱스 저장 후 origin데이터에서 유효 row만 추출해서 필터링함.
data_index = data_process.index.values.tolist()
data_filtered = data_origin.iloc[data_index,:]

# 앞 0~4열은 의미없는 정보이므로 삭제.
data_filtered = data_filtered.reset_index()
data_filtered = data_filtered.iloc[:,5:]


tmp_list = data_filtered["Unnamed: 4"].tolist()

data_tmp = []
idx = 0


# Unnamed: 4 칼럼 표시형식 통일화
for i, data in enumerate(data_filtered["Unnamed: 4"]):
    data = data.replace(' - ','-')
    if data == 'Formation#1 그리퍼접촉(%)':
        idx = i
        break
    data_tmp.append(data)

for data in tmp_list[idx:]:
    sp = data.find(' ')
    data = data[: sp] + '-' + data[sp + 1: ]

    data_tmp.append(data)

data_filtered["Unnamed: 4"] = data_tmp



공정 - 불량 데이터가 한열에 있으므로 공정 / 불량으로 나눔. 

In [360]:
data_process_and_loss = data_filtered['Unnamed: 4']
data_process_and_loss = data_process_and_loss.tolist()
data_filtered= data_filtered.drop('Unnamed: 4',axis = 1)
data_filtered = data_filtered.fillna(0)

In [361]:
# 사이트별로 공정/ loss 나눠주는 함수.
# input -> 공정 list
# output -> 공정 / loss / site 열을 갖고있는 Dataframe

def site_seperate(data_process_list):
    data_process_and_loss = data_process_list
    memo = ''
    site = ['Ochang', 'ESMI', 'ESNA', 'ESWA', 'ESNB']
    ret = pd.DataFrame()
    cursor = 0
    for data in data_process_and_loss:
        data_splited = data.split('-')

        # site별로 문자열 형식이 다름.
        if len(data_splited) == 2:
            process, loss = data_splited

        # 공백제거
        process = process.strip()
        loss = loss.strip()

        # 특성측정 후에 나오는 Formation은 다른 site
        if ('Formation' in process) and (memo == '특성측정'):
            cursor += 1

        memo = process
        mlist =  site[cursor], '활성화', process, loss



        ret = ret.append( pd.DataFrame([mlist]))



    return ret


In [362]:
data_temp = site_seperate(data_process_and_loss)


# 신규 데이터 처리
data_temp.index = [i for i in range(295)]
data_temp.columns = ['site','전체공정', '세부공정', '유형']
data_filtered['site'] = data_temp.iloc[:, 0]
data_filtered['전체공정'] = data_temp.iloc[:, 1]
data_filtered['세부공정'] = data_temp.iloc[:, 2]
data_filtered['유형'] = data_temp.iloc[:, 3]

# 열 순서 변경
column_list = data_filtered.columns.values.tolist()
column_list = column_list[26:] + column_list[:26]
data_filtered = data_filtered[column_list]



In [363]:

# 월 데이터 처리
column_month_list = []
for i in range(1,13):
    month = [str(i) + '월'] * 2
    month[0] += '_%'
    month[1] += '_억(원)'

    column_month_list += month
column_month_list += ['total_%','total_억(원)']

# 열이름 최종 업데이트
column_list[4:]  = column_month_list
data_filtered.columns = column_list


열 이름 리스트
['site', '전체공정', '세부공정', '유형',
 '1월_%', '1월_억(원)', '2월_%', '2월_억(원)',
 '3월_%', '3월_억(원)', '4월_%', '4월_억(원)',
 '5월_%', '5월_억(원)', '6월_%', '6월_억(원)',
 '7월_%', '7월_억(원)', '8월_%', '8월_억(원)',
 '9월_%', '9월_억(원)', '10월_%','10월_억(원)',
 '11월_%', '11월_억(원)', '12월_%', '12월_억(원)',
 'total_%', 'total_억(원)']

In [364]:
# 필요 칼럼만 수집.

data_filtered = data_filtered.loc[:, ['site', '전체공정', '세부공정', '유형',
                                      '1월_억(원)', '2월_억(원)', '3월_억(원)',
                                      '4월_억(원)', '5월_억(원)', '6월_억(원)',
                                      '7월_억(원)', '8월_억(원)', '9월_억(원)',
                                      '10월_억(원)', '11월_억(원)', '12월_억(원)', 'total_억(원)']]



data_final = pd.DataFrame()

for i in range(4,11):
    data_tmp = data_filtered.iloc[:,[0, 1, 2, 3, i]]
    column_name = data_tmp.columns.values.tolist()

    column_name[4] = 'Cost'
    data_tmp.columns = column_name
    data_tmp['month'] = str(i-3) + '월'

    data_final = pd.concat([data_final, data_tmp])

data_final.to_csv('output/활성화.csv', index= False, encoding='UTF-8-sig')
data_final

C:\ProgramData\Miniconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,site,전체공정,세부공정,유형,Cost,month
0,Ochang,활성화,Formation1,그리퍼접촉(%),0.134766,1월
1,Ochang,활성화,Formation1,충전전압(%),0.116852,1월
2,Ochang,활성화,Formation1,(Jig)가압단락 검사(%),0.164116,1월
3,Ochang,활성화,Formation1,외관변형 작업불가(%),0.095756,1월
4,Ochang,활성화,Formation1,J/F 온도(%),0.000000,1월
...,...,...,...,...,...,...
290,ESNB,활성화,특성측정,Taping(%),0.149945,7월
291,ESNB,활성화,특성측정,Sealing 주름(%),0.000000,7월
292,ESNB,활성화,특성측정,Cold IR(%),0.000000,7월
293,ESNB,활성화,특성측정,절연 저항(%),0.983421,7월
